Maria Musial 156062

# Computer vision - Lab 7

## Agenda

Image segmentation based on:
- thresholding
- cluster analysis,
- detecting image features (e.g. edges),
- region growing,



## Helpers

In [ ]:

%matplotlib inline
import glob

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import PIL
import plotly.graph_objects as go
import plotly.io as pio
from IPython.display import HTML, display
from matplotlib.colors import ListedColormap
from pandas import DataFrame
from sklearn.mixture import GaussianMixture

pd.options.display.html.border = 0
pd.options.display.float_format = '{:,.2f}'.format

### Images

*  Image Lenna (available at the [link](http://www.lenna.org/)) - one of the most popular images historically used for testing image processing and compression,
* clevr -comes from the CLEVR dataset that deals with the  Visual Query Answering problem,
* graf - sample graffiti image from the OpenCV repository OpenCV,
* sudoku - sample sudoku image from OpenCV repository,
* skittles - several images containing skittles

In [ ]:
# # download images
# !wget -O lena_std.tif http://www.lenna.org/lena_std.tif
# !wget -O clevr.jpg https://cs.stanford.edu/people/jcjohns/clevr/teaser.jpg
# !wget -O graf.png https://github.com/opencv/opencv/raw/master/samples/data/graf1.png
# !wget -O sudoku.png https://raw.githubusercontent.com/opencv/opencv/master/samples/data/sudoku.png

# for i in range(100, 111):
#     !wget -O skittles{i}.jpg https://github.com/possibly-wrong/skittles/blob/master/images/{i}.jpg?raw=true



### Visualization


In [ ]:
def imshow(a):
    a = a.clip(0, 255).astype("uint8")
    if a.ndim == 3:
        if a.shape[2] == 4:
            a = cv2.cvtColor(a, cv2.COLOR_BGRA2RGBA)
        else:
            a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
    display(PIL.Image.fromarray(a))

In [ ]:
css = """
<style type="text/css">
  table, td, table.dataframe, table.dataframe td {
    border: 1px solid black;    //border: double;
    border-collapse: collapse;
    border-style: solid;
    border-spacing: 0px;
    background-color: rgb(250,250,250);
    width: 18px;
    height: 18px;
    text-align: center;
    transform: scale(1.0);
    margin: 2px;
    }
</style>
"""


def h(s):
    return display(HTML(css + DataFrame(s).to_html(header=False, index=False)))

In [ ]:
def h_color(a, cmap="gray", scale=2):
    s = [a.shape[0] * scale, a.shape[1] * scale]
    plt.figure(figsize=s)
    plt.tick_params(
        axis="both",
        which="both",
        bottom=False,
        top=False,
        labelbottom=False,
        labelleft=False,
        left=False,
        right=False,
    )
    plt.imshow(a, cmap=cmap)

In [ ]:
cmap = ListedColormap(
    [
        "black",
        "tomato",
        "chocolate",
        "darkorange",
        "gold",
        "olive",
        "green",
        "deepskyblue",
        "blueviolet",
        "hotpink",
    ]
)


def h_grid(grid, scale=1):
    h_color(grid, cmap, scale)

In [ ]:
def pix_show(pixels, skip_each=1, height=400, width=400, colors=None):
    pixels = pixels[::skip_each]
    if colors is None:
        colors = pixels[:, ::-1]
    else:
        colors = colors[::skip_each]
    b, g, r = pixels[:, 0], pixels[:, 1], pixels[:, 2]
    fig = go.Figure(
        data=[
            go.Scatter3d(
                x=b,
                y=g,
                z=r,
                mode="markers",
                marker={"size": 2, "color": colors, "opacity": 0.7},
            )
        ],
        layout_xaxis_range=[0, 1],
        layout_yaxis_range=[0, 1],
    )
    scene = {
        "xaxis": dict(title="Blue"),
        "yaxis": dict(title="Green"),
        "zaxis": dict(title="Red"),
    }
    fig.update_layout(
        autosize=False, height=height, width=width, scene=scene, showlegend=True
    )
    pio.show(fig)

# Tasks

## Task 1

Like the section on multi-channel image segmentation, perform the same pixel intensity cluster analysis for the './skittles100.jpg' image and then segment the image using the K-Means algorithm (available, among others, in the scikit library: sklearn.cluster.KMeans) .

Present the intermediate results:
- BGR input image
- BGR pixels in 3D space,
- segmentation result on BGR pixels in 3D space,
- segmentation result as a 2D image (BGR)



In [ ]:
from sklearn.cluster import KMeans

skittles = cv2.imread("skittles100.jpg", cv2.IMREAD_COLOR)
imshow(cv2.resize(skittles, None, fx=0.4, fy=0.4))
skittles_pixels = skittles.reshape([-1, 3])
pix_show(skittles_pixels, 16)


Elbow method to get number of clusters that will work well with our segmentation later

In [ ]:

data = skittles_pixels
def elbow_method(data, k_range):
    inertia = [] # list to store the inertia values

    for k in k_range: # iterate over the range of k values
        kmeans = KMeans(n_clusters=k, n_init='auto') # create a KMeans instance with k clusters
        kmeans.fit(data) # fit the data to the KMeans instance
        inertia.append(kmeans.inertia_) # append the inertia value to the inertia list

    plt.figure(figsize=(8, 4))
    plt.plot(k_range, inertia, marker='o')
    plt.xlabel('Number of clusters (k)')
    plt.ylabel('Inertia')
    plt.title('Elbow Method for Optimal k')
    plt.grid(True)
    plt.show()

elbow_method(data, range(1, 12)) # call the elbow_method function with the data and a range of k values

In [ ]:
kmeans = KMeans(n_clusters=9, n_init='auto', random_state=42).fit(skittles_pixels)
segments = kmeans.predict(skittles_pixels)


segments_colors = np.stack([skittles_pixels[segments==i].mean(0) for i in range(9)], 0)  #get colors of each segment

colors = np.take(segments_colors, segments, 0)  #map colors to pixels
pix_show(skittles_pixels, 16, colors=colors[:, ::-1])

segmented = colors.reshape(skittles.shape)
imshow(cv2.resize(np.concatenate([skittles, segmented], 1), None, fx=0.4, fy=0.4))


## Task 2
Using the methods you learned in the previous class, find the number of Skittles in the image './skittles100.jpg'. (it is not necessary to use the solution from task 1)
**Show intermediate results and describe the processing steps in the comment.**

Show the original image with founded individual skittles marked on it.




#### How it works:
- get_segmented_explainable: read image, reshape to 1D array, fit KMeans, predict clusters, unify background, get colors of each cluster, map colors to pixels, reshape to image
    (I tried Gaussian blur, but it gave worse results.)
- count_skittles_explainable: binarizing image, deleting corners, closing in opening to separate skittles, growing circles back to skittles size, counting regions, getting edges of found skittles
- put_edges_explainable: putting the edges on the image, visualising results

All those functions have their version suited for task3, so that my computer doesnt explode when generating images and graphs.


In [ ]:
#read image, reshape to 1D array, fit KMeans, predict clusters, unify background, get colors of each cluster, map colors to pixels, reshape to image
    #Gaussian blur gave worse results.
    
def get_segmented_explainable(path):
    skittles = cv2.imread(path, cv2.IMREAD_COLOR)
    imshow(cv2.resize(skittles, None, fx=0.4, fy=0.4))
    skittles_pixels = skittles.reshape([-1, 3])
    print("Pixels in RGB space:")
    pix_show(skittles_pixels, 16)
    kmeans = KMeans(n_clusters=9, n_init='auto', random_state=42).fit(skittles_pixels)
    segments = kmeans.predict(skittles_pixels)
    segments_colors = np.stack([skittles_pixels[segments==i].mean(0) for i in range(9)], 0)  #get colors of each segment

    colors = np.take(segments_colors, segments, 0)  #map colors to pixels
    print("Pixels clustered:")
    pix_show(skittles_pixels, 16, colors=colors[:, ::-1])

    segmented = colors.reshape(skittles.shape)
    print("Segmented image:")
    imshow(cv2.resize(np.concatenate([skittles, segmented], 1), None, fx=0.4, fy=0.4))
    cols=[]
    for color in segments_colors:
        cols.append(np.full((100,100,3),color))
    imshow(cv2.resize(np.concatenate(cols, 1), None, fx=0.4, fy=0.4))

    #I want to unify background, get all grey segments and reassign values to one of greys
    grey_segments = []
    grey_color = np.array([128, 128, 128])
    for i, color in enumerate(segments_colors):
        if np.allclose(color, grey_color, atol=100):
            grey_segments.append(i)  
    print("Background clusters:", grey_segments)
    segments[np.isin(segments, grey_segments)] = grey_segments[0]  # get background to be one

    segments_colors = []
    for i in range(9):
        segment_pixels = skittles_pixels[segments == i]
        if segment_pixels.size > 0:
            segment_mean = segment_pixels.mean(0)
            segments_colors.append(segment_mean)
        else:
            segments_colors.append(np.array([128, 128, 128], dtype=np.uint8))  # Default grey color for empty segments
    segments_colors = np.array(segments_colors, dtype=np.uint8)

    colors = np.take(segments_colors, segments, 0)  #map colors to pixels

    segmented = colors.reshape(skittles.shape)
    print("Segmented image with unified background:")
    imshow(cv2.resize(np.concatenate([skittles, segmented], 1), None, fx=0.4, fy=0.4))
    return segmented


#binarizing image, deleting corners, closing in opening to separate skittles, growing circles back to skittles size, counting regions, getting edges of found skittles, visualising results
def count_skittles_explainable(image):
    #WHAT ARE WE WORKING ON
    bin_skit = (image[:, :, 0] > 100).astype(np.uint8) * 255
    bin_skit = cv2.bitwise_not(bin_skit)
    print("The image we're counting skittles on:")
    imshow(cv2.resize(image, None, fx=0.4, fy=0.4))

    print("The binarized version:")
    imshow(cv2.resize(bin_skit, None, fx=0.4, fy=0.4))


    #Deleting corners
    height, width = bin_skit.shape
    corner_size = 50  # Size of the corner regions to mask
    mask = np.full(bin_skit.shape, 255, dtype=np.uint8)
    corner_positions = [
        (0, 0), 
        (width - corner_size, 0), 
        (0, height - corner_size),  
        (width - corner_size, height - corner_size)  
    ]
    for x, y in corner_positions:
        cv2.rectangle(mask, (x, y), (x + corner_size, y + corner_size), 0, -1)
    bin_skit[mask == 0] = 0
    print("Deleted corners, to focus on skittles:")
    imshow(cv2.resize(bin_skit, None, fx=0.4, fy=0.4))


    #CLOSING IN OPENING TO SEPARATE SKITTLES
    struct = np.ones((3, 3), np.uint8)

    clos = cv2.dilate(bin_skit, struct, iterations=4)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))  
    img_bin_dil = cv2.erode(clos, struct, iterations=13)
    img_bin_dil_ker = cv2.erode(img_bin_dil, kernel, iterations=16)             #right number of skittles!!!!

    print("Closing to close 's' on skittles, erosion to separate them to be able to count them")
    imshow(cv2.resize(np.concatenate([img_bin_dil, img_bin_dil_ker],1), None, fx=0.4, fy=0.4))
    
    num_labels, labels = cv2.connectedComponents(img_bin_dil_ker)
    print(f"Number of regions (skittles): {num_labels - 1}")


    #Grow circles back to skittles size
    kernel = np.ones((30,30), np.uint8)  
    grown_regions = np.zeros_like(img_bin_dil_ker, dtype=np.uint8)
    for label in range(1, num_labels):  
        label_mask = np.uint8(labels == label) * 255
        dilated_mask = cv2.dilate(label_mask, kernel, iterations=1)
        grown_regions = cv2.bitwise_or(grown_regions, dilated_mask)
        
    edges = cv2.Canny(grown_regions, threshold1=100, threshold2=200)
    print("Growing regions back to skittles size, edges:")
    imshow(cv2.resize(np.concatenate([img_bin_dil_ker, grown_regions, edges], 1), None, fx=0.4, fy=0.4))  
    return num_labels - 1, edges


def put_edges_exp(path):
    skittles, edges = count_skittles_explainable(get_segmented_explainable(path))    
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    edges_rgb = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
    imshow(cv2.resize(np.concatenate([image, edges_rgb], 1), None, fx=0.4, fy=0.4))
    edges_mask = edges_rgb == 255
    image[edges_mask] = 255
    return image

#VISUALISING THE RESULTS FOR TASK3, just image->results
def count_skittles(image):
    #WHAT ARE WE WORKING ON
    bin_skit = (image[:, :, 0] > 100).astype(np.uint8) * 255
    bin_skit = cv2.bitwise_not(bin_skit)

    #Deleting corners
    height, width = bin_skit.shape
    corner_size = 50  # Size of the corner regions to mask
    mask = np.full(bin_skit.shape, 255, dtype=np.uint8)
    corner_positions = [
        (0, 0), 
        (width - corner_size, 0), 
        (0, height - corner_size),  
        (width - corner_size, height - corner_size)  
    ]
    for x, y in corner_positions:
        cv2.rectangle(mask, (x, y), (x + corner_size, y + corner_size), 0, -1)
    bin_skit[mask == 0] = 0


    #CLOSING IN OPENING TO SEPARATE SKITTLES
    struct = np.ones((3, 3), np.uint8)

    clos = cv2.dilate(bin_skit, struct, iterations=4)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))  
    img_bin_dil = cv2.erode(clos, struct, iterations=13)
    img_bin_dil_ker = cv2.erode(img_bin_dil, kernel, iterations=16)             #right number of skittles!!!!
    
    num_labels, labels = cv2.connectedComponents(img_bin_dil_ker)

    #Grow circles back to skittles size
    kernel = np.ones((30,30), np.uint8)  
    grown_regions = np.zeros_like(img_bin_dil_ker, dtype=np.uint8)
    for label in range(1, num_labels):  
        label_mask = np.uint8(labels == label) * 255
        dilated_mask = cv2.dilate(label_mask, kernel, iterations=1)
        grown_regions = cv2.bitwise_or(grown_regions, dilated_mask)
        
    edges = cv2.Canny(grown_regions, threshold1=100, threshold2=200)
    return num_labels - 1, edges

def get_segmented(path):
    skittles = cv2.imread(path, cv2.IMREAD_COLOR)
    skittles_pixels = skittles.reshape([-1, 3])
    kmeans = KMeans(n_clusters=9, n_init='auto', random_state=42).fit(skittles_pixels)
    segments = kmeans.predict(skittles_pixels)
    segments_colors = np.stack([skittles_pixels[segments==i].mean(0) for i in range(9)], 0)  #get colors of each segment

    colors = np.take(segments_colors, segments, 0)  #map colors to pixels

    segmented = colors.reshape(skittles.shape)
    cols=[]
    for color in segments_colors:
        cols.append(np.full((100,100,3),color))

    #I want to unify background
    grey_segments = []
    grey_color = np.array([128, 128, 128])
    for i, color in enumerate(segments_colors):
        if np.allclose(color, grey_color, atol=100):
            grey_segments.append(i)  
    segments[np.isin(segments, grey_segments)] = grey_segments[0]  # get background to be one

    segments_colors = []
    for i in range(9):
        segment_pixels = skittles_pixels[segments == i]
        if segment_pixels.size > 0:
            segment_mean = segment_pixels.mean(0)
            segments_colors.append(segment_mean)
        else:
            segments_colors.append(np.array([128, 128, 128], dtype=np.uint8))  # Default grey color for empty segments
    segments_colors = np.array(segments_colors, dtype=np.uint8)

    colors = np.take(segments_colors, segments, 0)  #map colors to pixels

    segmented = colors.reshape(skittles.shape)
    return segmented

def put_edges_task3(path):
    skittles, edges = count_skittles(get_segmented(path))    
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image_t = image.copy()
    edges_rgb = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
    edges_mask = edges_rgb == 255
    image_t[edges_mask] = 255
    imshow(cv2.resize(np.concatenate([image, image_t],1), None, fx=0.4, fy=0.4))


imshow(cv2.resize(put_edges_exp("skittles100.jpg"), None, fx=0.4, fy=0.4))

## Task 3
1. Test the solution from task 2 for the remaining skittels images.
2. Improve the solution so that it works properly for this images.

In [ ]:
for file in glob.glob("./skittles*"):
    print(file)
    skittles = cv2.imread(file, cv2.IMREAD_COLOR)
    # imshow(cv2.resize(skittles, None, fx=0.4, fy=0.4))
    put_edges_task3(file)


# I improved the erosion and closing, so that it gets more skittles right. 
# Problem is with cluster of yellows in 109, and images where flash+"s" is quite big on skittles. 
# Noise in 103 (clump of skittles?) is dealt with.
#Overall results are +/- 4 skittles. 